In [117]:
#Importing packages
!pip install tweepy
!pip install wget
!pip install mysql-connector-python
import tweepy
import pandas as pd
import mysql.connector

In [118]:
#tweepy api authentication keys
consumer_key = "ZdINpkFqjeH4aDt1vSA1ChGrJ"
consumer_secret = "fACYEj7eznUvZ57PcDaPbLGSwd5RDaF0Vj3YdVamh1lOBrrE12"
access_token = "935762631029379072-M8hBh2cJxijxwzyUUnFlXNysYFhpP0t"
access_token_secret = "PbiGTho0VdGJyIk3InFzUhHEx3KNH9YDAVPEHi0In6mjB"


In [119]:
# connecting to tweepy api
auth = tweepy.OAuth1UserHandler(
   consumer_key, consumer_secret, access_token, access_token_secret
)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [120]:
# defining variables for search
new_search = "#Yelp"
num_tweets = 10
hashtag_tweets = []
user_info = []

In [121]:
#scraping twitter to get most recent tweets that have "#Socialize" 
tweets = tweepy.Cursor(api.search_tweets,q='#Yelp',count=num_tweets,lang="en",tweet_mode='extended')
#extracting the tweet_info and user_info of all "#yelp" tweets
for tweet in tweets.items(10):
    if not tweet.entities["urls"]:
        continue
    else:
        hashtag_tweets.append((tweet.id, tweet.user.id, tweet.full_text, tweet.entities["urls"][0]["url"],tweet.retweet_count, tweet.favorite_count, tweet.created_at))
        user_info.append((tweet.user.id, tweet.user.screen_name, tweet.user.location, tweet.user.created_at))
       
    

In [122]:
#creating a dataframe to store user information
df_user = pd.DataFrame(user_info, columns =['user_id', 'username', 'location', 'created_at'])
df_user


,user_id,username,location,created_at
0,85278330,BigDfrmLV,N 36°1' 0'' / W 115°7' 0'',2009-10-26 09:39:20+00:00
1,27832688,4magician,San Diego,2009-03-31 07:28:14+00:00
2,1152851226,aj6079,"Downey, CA",2013-02-06 04:12:18+00:00
3,3043901,kimdc,Hawaii ☀,2007-03-31 06:52:19+00:00
4,1494402864,greyseahorsey,USA,2013-06-09 02:09:22+00:00
5,424346533,TimBlackDragon,"Los Angeles, CA",2011-11-29 17:22:39+00:00
6,4763733149,markpinelife,"New York, NY",2016-01-10 11:24:13+00:00
7,16731234,Gaponte,"US: 41.078301,-73.868445",2008-10-14 01:21:27+00:00
8,597649979,fdoug23,,2012-06-02 18:50:16+00:00
9,59572970,wawahwest,Green Bubbles on ya iPhone,2009-07-23 20:06:13+00:00


In [123]:
#creating a dataframe to store "#Socialize" tweets information
df_hash = pd.DataFrame(hashtag_tweets, columns =['tweet_id', 'user_id', 'text', 'url','retweets','likes','time'])
df_hash

,tweet_id,user_id,text,url,retweets,likes,time
0,1594868479215427585,85278330,I checked in at Las Vegas Paiute Tribe on #Yel...,https://t.co/SgEp70Llf6,0,0,2022-11-22 01:40:51+00:00
1,1594866513840050177,27832688,I checked in at McDonald's on #Yelp https://t....,https://t.co/2lUjZR9Iww,0,0,2022-11-22 01:33:02+00:00
2,1594865315980382209,1152851226,Dinner (@ Pieloon) on #Yelp https://t.co/k7grZ...,https://t.co/k7grZksSsk,0,0,2022-11-22 01:28:17+00:00
3,1594864671986941952,3043901,I checked in at Trader Joe's on #Yelp https://...,https://t.co/2WjHwsfCSx,0,0,2022-11-22 01:25:43+00:00
4,1594861602217811969,1494402864,Shopping with my husband. Had keys made by Mac...,https://t.co/dIXNkk5OcQ,0,0,2022-11-22 01:13:31+00:00
5,1594861416003293184,424346533,I checked in at Venice Canals on #Yelp https:/...,https://t.co/O97WNvX19O,0,0,2022-11-22 01:12:47+00:00
6,1594861143193161728,4763733149,I checked in at CVS Pharmacy on #Yelp https://...,https://t.co/UBfW79wWiq,0,0,2022-11-22 01:11:42+00:00
7,1594856323262578689,16731234,I checked in at Quisqueya Restaurant on #Yelp ...,https://t.co/zu3PurgiPF,0,0,2022-11-22 00:52:33+00:00
8,1594855922538811392,597649979,I checked in at Birds of a Feather on #Yelp ht...,https://t.co/PExoaTnczv,0,0,2022-11-22 00:50:57+00:00
9,1594854170913542145,59572970,Adulting pt. 2 (@ The Home Depot) on #Yelp htt...,https://t.co/P69jfCfFxi,0,0,2022-11-22 00:43:59+00:00


In [124]:
#making a list of user_id in df_user
df_user_id = df_user["user_id"].tolist()

print (df_user_id)

[85278330, 27832688, 1152851226, 3043901, 1494402864, 424346533, 4763733149, 16731234, 597649979, 59572970]


In [128]:
recent_tweets = []

In [127]:
#extracting the tweets of all users in df_user table
for user in df_user_id:
    rcnt_tweets = tweepy.Cursor(api.user_timeline,user_id=user,count=20,tweet_mode='extended')
    for tweet in rcnt_tweets.items(20):
        recent_tweets.append((tweet.id, tweet.user.id, tweet.full_text, tweet.created_at))

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded

In [92]:
#creating a dataframe of user_timeline
df_user_tweets = pd.DataFrame(recent_tweets, columns =['tweet_id','user_id', 'text', 'created_at'])
df_user_tweets

,tweet_id,user_id,text,created_at
0,1594861602217811969,1494402864,Shopping with my husband. Had keys made by Mac...,2022-11-22 01:13:31+00:00
1,1594087791293616132,1494402864,"Seen ""Pepsi, Where's My Jet?"" on Netflix yet?\...",2022-11-19 21:58:40+00:00
2,1593501441431314433,1494402864,RT @OKCPD: This man yelled a homophobic slur b...,2022-11-18 07:08:44+00:00
3,1593452212063244293,1494402864,https://t.co/6Et0w1zjTM,2022-11-18 03:53:06+00:00
4,1593339008620322816,1494402864,3 Ingredient Meltaway Cookies (No Flour or Egg...,2022-11-17 20:23:17+00:00
...,...,...,...,...
9695,1570971749273186307,528986937,We got the best 👌 #desserts #cakes #pastries #...,2022-09-17 03:03:46+00:00
9696,1570656795357749248,528986937,"Just posted a photo @ Natas Pastries, LA's Por...",2022-09-16 06:12:16+00:00
9697,1566407357051142149,528986937,Bom dia 🎂\n#nataspastries #larestaurants #lafo...,2022-09-04 12:46:31+00:00
9698,1563646258715447296,528986937,Francesinha 🥪 #traditional #sandwich #portugue...,2022-08-27 21:54:53+00:00


In [93]:
df_user_tweets

,tweet_id,user_id,text,created_at
0,1594861602217811969,1494402864,Shopping with my husband. Had keys made by Mac...,2022-11-22 01:13:31+00:00
1,1594087791293616132,1494402864,"Seen ""Pepsi, Where's My Jet?"" on Netflix yet?\...",2022-11-19 21:58:40+00:00
2,1593501441431314433,1494402864,RT @OKCPD: This man yelled a homophobic slur b...,2022-11-18 07:08:44+00:00
3,1593452212063244293,1494402864,https://t.co/6Et0w1zjTM,2022-11-18 03:53:06+00:00
4,1593339008620322816,1494402864,3 Ingredient Meltaway Cookies (No Flour or Egg...,2022-11-17 20:23:17+00:00
...,...,...,...,...
9695,1570971749273186307,528986937,We got the best 👌 #desserts #cakes #pastries #...,2022-09-17 03:03:46+00:00
9696,1570656795357749248,528986937,"Just posted a photo @ Natas Pastries, LA's Por...",2022-09-16 06:12:16+00:00
9697,1566407357051142149,528986937,Bom dia 🎂\n#nataspastries #larestaurants #lafo...,2022-09-04 12:46:31+00:00
9698,1563646258715447296,528986937,Francesinha 🥪 #traditional #sandwich #portugue...,2022-08-27 21:54:53+00:00


In [76]:
#connect to mysql servel
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="1965",
  database="testdb"
)

print(mydb)

mycursor = mydb.cursor()


In [77]:
# create users table and adding contraints
mycursor.execute("DROP TABLE IF EXISTS users")
mycursor.execute("CREATE TABLE users (user_id VARCHAR(255) PRIMARY KEY, username VARCHAR(255), location VARCHAR(255), created_at TIMESTAMP)")

DatabaseError: 3730 (HY000): Cannot drop table 'users' referenced by a foreign key constraint 'user_id_fk' on table 'hashtag'.

In [ ]:
# create socialize_account table and adding contraints
mycursor.execute("DROP TABLE IF EXISTS socialize_account")
mycursor.execute("CREATE TABLE socialize_account (user_id VARCHAR(255) PRIMARY KEY, username VARCHAR(255))")

In [ ]:
# inserting df_user dataframe in sql user table 
val = list(df_user.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO users (user_id, username, location, created_at) VALUES (%s, %s, %s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

In [ ]:
# inserting df_socialize_account dataframe in mysql table 
val = list(df_user.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO socialize_account (user_id, username) VALUES (%s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

In [22]:
# create tweet_mention table and adding constraints
mycursor.execute("DROP TABLE IF EXISTS tweet_mention")
mycursor.execute("CREATE TABLE tweet_mention (tweet_id VARCHAR(255) PRIMARY KEY, user_id VARCHAR(255), text TEXT(6553), url VARCHAR(255), retweets INT, likes INT, time TIMESTAMP, CONSTRAINT user_id_fk FOREIGN KEY (user_id) REFERENCES users (user_id) ON UPDATE CASCADE ON DELETE CASCADE)")



In [23]:
# inserting df_hash dataframe in sql table 
val = list(df_hash.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO tweet_mention (tweet_id, user_id, text, url, retweets, likes, time) VALUES (%s, %s, %s, %s, %s, %s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

485 was inserted.


In [24]:
# create user tweet table and adding constraints
mycursor.execute("DROP TABLE IF EXISTS tweet")
mycursor.execute("CREATE TABLE tweet (tweet_id VARCHAR(255), user_id VARCHAR(255), text TEXT(6553), created_at TIMESTAMP, CONSTRAINT past_tweets PRIMARY KEY(tweet_id,user_id), CONSTRAINT timeline_user_fk FOREIGN KEY (user_id) REFERENCES users (user_id) ON UPDATE CASCADE ON DELETE CASCADE)")

In [25]:
# inserting user_tweets dataframe in sql timeline table 
val = list(df_user_tweets.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO tweet (tweet_id, user_id, text, created_at) VALUES (%s, %s, %s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

4881 was inserted.
